# P4 Regularización, Normalización y Aumentado de datos



In [1]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

11490434/11490434 [==============================] - 0s 0us/step
training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


## Modelo base
 Partiremos de una topología base e iremos añadiendo diferentes estrategias de regularización para mejorar el rendimiento del modelo.



In [2]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
373/375 [============================>.] - ETA: 0s - loss: 0.3466 - accuracy: 0.8989
Epoch 1: val_accuracy improved from -inf to 0.94942, saving model to best_model.h5
375/375 [==============================] - 13s 11ms/step - loss: 0.3458 - accuracy: 0.8991 - val_loss: 0.1714 - val_accuracy: 0.9494 - lr: 0.0250
Epoch 2/25
  9/375 [..............................] - ETA: 2s - loss: 0.1428 - accuracy: 0.9592

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


374/375 [============================>.] - ETA: 0s - loss: 0.1363 - accuracy: 0.9607
Epoch 2: val_accuracy improved from 0.94942 to 0.96200, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.1362 - accuracy: 0.9608 - val_loss: 0.1215 - val_accuracy: 0.9620 - lr: 0.0250
Epoch 3/25
365/375 [============================>.] - ETA: 0s - loss: 0.0904 - accuracy: 0.9732
Epoch 3: val_accuracy improved from 0.96200 to 0.96858, saving model to best_model.h5
375/375 [==============================] - 3s 9ms/step - loss: 0.0908 - accuracy: 0.9730 - val_loss: 0.1070 - val_accuracy: 0.9686 - lr: 0.0250
Epoch 4/25
369/375 [============================>.] - ETA: 0s - loss: 0.0642 - accuracy: 0.9819
Epoch 4: val_accuracy improved from 0.96858 to 0.97350, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.0640 - accuracy: 0.9820 - val_loss: 0.0901 - val_accuracy: 0.9735 - lr: 0.0250
Epoch 5/25
370/375 [===========

## Regularización l2 (o l1)

La regularización l2 consiste en añadir a la función de coste una penalización proporcional a la norma l2 de los pesos del modelo. De esta forma, se penaliza a los pesos que tengan un valor alto, forzando a que los pesos tengan valores pequeños. Esto se conoce como regularización l2. También podríamos hacer lo mismo con regularización l1 o con ambas (lo que se conoce como *Elastic net*)


In [7]:
## Teniendo en cuenta el modelo base añade regularización L2 a las capas densas
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2,l1

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.01))) #el kernel regulariza sobre los pesos no sobre el bias, el 0,01 es el lambda -> Loss + lambda ||W||2 2
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.01))) #cambiando l2 por l1 cambias de regularizacion
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
368/375 [============================>.] - ETA: 0s - loss: 5.9437 - accuracy: 0.8849
Epoch 1: val_accuracy improved from -inf to 0.90450, saving model to best_model.h5
375/375 [==============================] - 3s 5ms/step - loss: 5.8503 - accuracy: 0.8854 - val_loss: 0.9587 - val_accuracy: 0.9045 - lr: 0.0250
Epoch 2/25
 29/375 [=>............................] - ETA: 1s - loss: 0.8522 - accuracy: 0.9278

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


369/375 [============================>.] - ETA: 0s - loss: 0.5764 - accuracy: 0.9302
Epoch 2: val_accuracy improved from 0.90450 to 0.93900, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.5746 - accuracy: 0.9304 - val_loss: 0.4452 - val_accuracy: 0.9390 - lr: 0.0250
Epoch 3/25
363/375 [============================>.] - ETA: 0s - loss: 0.4290 - accuracy: 0.9393
Epoch 3: val_accuracy did not improve from 0.93900
375/375 [==============================] - 2s 4ms/step - loss: 0.4295 - accuracy: 0.9391 - val_loss: 0.4306 - val_accuracy: 0.9362 - lr: 0.0250
Epoch 4/25
362/375 [===========================>..] - ETA: 0s - loss: 0.4051 - accuracy: 0.9430
Epoch 4: val_accuracy did not improve from 0.93900
375/375 [==============================] - 2s 5ms/step - loss: 0.4044 - accuracy: 0.9432 - val_loss: 0.4091 - val_accuracy: 0.9379 - lr: 0.0250
Epoch 5/25
366/375 [============================>.] - ETA: 0s - loss: 0.3869 - accuracy: 0.9464
Epoch 5

## Dropout

El dropout es una técnica de regularización que consiste en eliminar aleatoriamente un porcentaje de las neuronas de la red durante el entrenamiento. De esta forma, se evita que la red se sobreajuste a los datos de entrenamiento y se mejora la generalización del modelo.


El dropout se carga el valor de la neurona con cierta probabilidad, entonces el valor que tenia pasa a ser cero si el dropout es de p=0.5, con probabilidad 0.5 cada neurona se pondra a 0 o mantendra su valor, se hace de manera aleatoria

Con esto evitas que alguna neurona se superespecialice, evita el overfitting y se acerca a la generalizacion.

en fase de entrenamiento se hace como esta aplciado arriba
en fase inferencia todas las neuronas estan activas y para que los vlaores no sean excesivamente grandes, se multiplica cada de uno de los pesos por 1-p para reajustarlo.

se hace con un if, si esta en entrenamiento aplicas dropout, sino no

In [8]:
## Teniendo en cuenta el modelo base añade regularización de tipo dropout a las capas densas
from keras import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/25
373/375 [============================>.] - ETA: 0s - loss: 0.4548 - accuracy: 0.8602
Epoch 1: val_accuracy improved from -inf to 0.94650, saving model to best_model.h5
375/375 [==============================] - 5s 7ms/step - loss: 0.4537 - accuracy: 0.8606 - val_loss: 0.1844 - val_accuracy: 0.9465 - lr: 0.0250
Epoch 2/25
371/375 [============================>.] - ETA: 0s - loss: 0.1971 - accuracy: 0.9404
Epoch 2: val_accuracy improved from 0.94650 to 0.96233, saving model to best_model.h5
375/375 [==============================] - 3s 7ms/step - loss: 0.1967 - accuracy: 0.9405 - val_loss: 0.1263 - val_accuracy: 0.9623 - lr: 0.0250
Epoch 3/25
369/375 [============================>.] - ETA: 0s - loss: 0.1465 - accuracy: 0.9560
Epoch 3: val_accuracy improved from 0.96233 to 0.96867, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.1461 - accuracy: 0.9560 - val_loss: 0.1059 - val_accuracy: 0.9687 - lr: 0.0250
Epoch 4/25
368/375 [=====

## Normalización BatchNorm

La normalización BatchNorm consiste en normalizar la salida de una capa de la red neuronal para que tenga media 0 y varianza 1. De esta forma, se consigue que la red neuronal pueda entrenarse más rápido y que sea más robusta a cambios en los pesos de las capas anteriores.


Al normalizar todos los datos q entran en la siguiente capa entran con media 0 y desviacion tipica 1. la normalizacion se hace para cada valor, se resta la media del batch y se divide por la desviacion tipica de es batch. Al dividir le sumas epsilon en el dividendo para q no divida por 0.


!!!!Es muy buena practica noramlizar los datos despues de cada capa, asi se puede manejar de manera eficiente los datos.  Con batch pequeño puede dar problemas y hoy en dia se usa mas el layernorm

In [9]:
## Teniendo en cuenta el modelo base añade normalización BatchNorm
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())#aqui igual, los argumentos pueden ser gamma y beta, que transforma gamma*X + beta
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())#con esto ya haces la normalizacion
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/25
373/375 [============================>.] - ETA: 0s - loss: 0.2080 - accuracy: 0.9413
Epoch 1: val_accuracy improved from -inf to 0.96225, saving model to best_model.h5
375/375 [==============================] - 4s 6ms/step - loss: 0.2078 - accuracy: 0.9414 - val_loss: 0.1284 - val_accuracy: 0.9622 - lr: 0.0250
Epoch 2/25
369/375 [============================>.] - ETA: 0s - loss: 0.0642 - accuracy: 0.9796
Epoch 2: val_accuracy improved from 0.96225 to 0.97367, saving model to best_model.h5
375/375 [==============================] - 3s 7ms/step - loss: 0.0642 - accuracy: 0.9797 - val_loss: 0.0881 - val_accuracy: 0.9737 - lr: 0.0250
Epoch 3/25
368/375 [============================>.] - ETA: 0s - loss: 0.0363 - accuracy: 0.9883
Epoch 3: val_accuracy improved from 0.97367 to 0.97375, saving model to best_model.h5
375/375 [==============================] - 2s 7ms/step - loss: 0.0360 - accuracy: 0.9884 - val_loss: 0.0970 - val_accuracy: 0.9737 - lr: 0.0250
Epoch 4/25
363/375 [=====

## Aumentado de datos

El aumentado de datos consiste en generar nuevos datos de entrenamiento a partir de los datos de entrenamiento originales. De esta forma, se consigue que el modelo sea más robusto y que se generalice mejor a datos que no ha visto durante el entrenamiento.

En nuestro caso para los dígitos de la MNIST vamos a realizar un aumento de datos de la siguiente forma:

- Rotación aleatoria de la imagen entre -30 y 30 grados.
- Traslación aleatoria de la imagen entre -3 y 3 píxeles en horizontal y vertical.
- Escalado aleatorio de la imagen entre 0.8 y 1.2.
- Inversión aleatoria de la imagen en horizontal y vertical. **NO!!!** depende de la tarea en imagenes de animales puede servir.

El aumentado de datos se ejecuta en CPU y ralentiza el entrenamiento.

Normalmente además, se necesitarán más epochs para entrenar el modelo.



Para añadir ruido a la imagen se puede hacer añadiendo gaussian noise en la capa de entrada de datos
Cut out, sirve para tapar una parte de la imagen. asi forzar a la red a aprender.
Puedes hacer combinaciones lineales con dos imagenes, mixup con lo que la probabilidad q sea coche y vaca sera porporcional al mix que hayas hecho

Tambien se puede cargar palabras, o transponer algunas.

In [10]:
## Implementamos en el ejemplo base el aumentado de datos
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization,Reshape
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30, #de -30 a 30
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,# de -0.8 a 1.2
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest') #cuando lo haces pequeño lo rellenas con lo mas cercano

## Importante: ImageDataGenerator espera una imagen con 3 canales, necesitamos hacer reshape
x_train = x_train.reshape(48000, 28, 28, 1) #con un canal, sino no funciona, modo de gris
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## Ajustamos el generador de datos
datagen.fit(x_train) #lo calculas si a los datos de entrada le metes alguna transformacion estadistica

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input((28,28,1)))#despues de esto se aplica de forma automatica el image data generator
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
## Entrenamos con el generador de datos en lugar de con el dataset
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),#el .flow sirve para analizar imagenes generadas, es un flujo de imagenes, la cpu se encarga de las modificaciones
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 0.6048 - accuracy: 0.8189
Epoch 1: val_accuracy improved from -inf to 0.94150, saving model to best_model.h5
375/375 [==============================] - 24s 60ms/step - loss: 0.6048 - accuracy: 0.8189 - val_loss: 0.1890 - val_accuracy: 0.9415 - lr: 0.0250


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/25
374/375 [============================>.] - ETA: 0s - loss: 0.3402 - accuracy: 0.8982
Epoch 2: val_accuracy improved from 0.94150 to 0.94942, saving model to best_model.h5
375/375 [==============================] - 17s 46ms/step - loss: 0.3400 - accuracy: 0.8983 - val_loss: 0.1712 - val_accuracy: 0.9494 - lr: 0.0250
Epoch 3/25
374/375 [============================>.] - ETA: 0s - loss: 0.2794 - accuracy: 0.9154
Epoch 3: val_accuracy improved from 0.94942 to 0.95000, saving model to best_model.h5
375/375 [==============================] - 18s 47ms/step - loss: 0.2789 - accuracy: 0.9155 - val_loss: 0.1573 - val_accuracy: 0.9500 - lr: 0.0250
Epoch 4/25
374/375 [============================>.] - ETA: 0s - loss: 0.2421 - accuracy: 0.9289
Epoch 4: val_accuracy improved from 0.95000 to 0.95833, saving model to best_model.h5
375/375 [==============================] - 17s 46ms/step - loss: 0.2420 - accuracy: 0.9289 - val_loss: 0.1333 - val_accuracy: 0.9583 - lr: 0.0250
Epoch 5/25
375/3

KeyboardInterrupt: ignored

## Ejercicio:

Probar todas las técnicas presentadas para obtener un acierto en **test > 99%**.

Se aconseja no malgastar datos de entrenamiento y por lo tanto emplear todo el training set para el entrenamiento. No emplear conjunto de validación y emplear el test set al final para calcular el acierto.

A modo de "trampa" podríamos ejecutar el fit con los datos de test en validation_data para así monitorizar si llegamos a ese 99%

validation_data=(x_test, y_test)
